In [ ]:
!pip install rdflib

   ---------------------------------------- 0.0/564.9 kB ? eta -:--:--
   ------------------ --------------------- 262.1/564.9 kB ? eta -:--:--
   ---------------------------------------- 564.9/564.9 kB 2.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from rdflib import Graph, URIRef, Literal
# Load the Turtle file
g = Graph()
g.parse("filtered_knowledge_poi_no_trees_no_bench_no_tables.ttl", format="turtle")

# Explore the graph size
print(f"Graph has {len(g)} triples.")

Graph has 76981 triples.


In [ ]:
from collections import Counter

filtered_graph = g
# Define the prefix you want to match for the object
prefix = "http://poimilano.disco.unimib.it/class/"

# Collect all objects that start with the specified prefix
matching_objects = [o for s, p, o in filtered_graph if isinstance(o, URIRef) and str(o).startswith(prefix)]

# Count the occurrences of each unique object
object_counts = Counter(matching_objects)

# Sort the counts in descending order
sorted_counts = object_counts.most_common()

# Print the count for each unique object in descending order
for obj, count in sorted_counts:
    print(f"Object: {obj} | Count: {count}")

In [ ]:
from collections import Counter
from rdflib import URIRef

# Define the prefix and target predicate
prefix = "http://poimilano.disco.unimib.it/class/"
target_predicate = URIRef("http://www.wikidata.org/prop/direct/P31")

# Collect all objects that start with the specified prefix and have the target predicate
matching_objects = [
    o for s, p, o in filtered_graph
    if p == target_predicate and isinstance(o, URIRef) and str(o).startswith(prefix)
]

# Count the occurrences of each unique object
object_counts = Counter(matching_objects)

# Sort the counts in descending order
sorted_counts = object_counts.most_common()

# Print the count for each unique object in descending order
for obj, count in sorted_counts:
    print(f"Object: {obj} | Count: {count}")


In [ ]:
# Define the target predicate and objects
target_predicate = URIRef("http://www.wikidata.org/prop/direct/P31")
target_objects = {
    #URIRef("http://poimilano.disco.unimib.it/class/ParcoEPiazza"),
    #URIRef("http://poimilano.disco.unimib.it/class/Biblioteca"),
    URIRef("http://poimilano.disco.unimib.it/class/FermataDellaMetropolitana"),
    URIRef("http://poimilano.disco.unimib.it/class/Museo"),
    URIRef("http://poimilano.disco.unimib.it/class/Teatro"),
    URIRef("http://poimilano.disco.unimib.it/class/MonumentoCommemorativo"),
    URIRef("http://poimilano.disco.unimib.it/class/Monumento"),
    URIRef("http://poimilano.disco.unimib.it/class/StazioneSotterranea"),
    URIRef("http://poimilano.disco.unimib.it/class/TargaCommemorativa"),
    #URIRef("http://poimilano.disco.unimib.it/class/SalaDegliArchivi"),
    URIRef("http://poimilano.disco.unimib.it/class/Scultura"),
    URIRef("http://poimilano.disco.unimib.it/class/Statua"),
    URIRef("http://poimilano.disco.unimib.it/class/PiantaMonumentale")
}

# Step 1: Identify subjects to keep
subjects_to_keep = {
    s for s, p, o in filtered_graph if p == target_predicate and o in target_objects
}

# Step 2: Create a filtered graph with only relevant subjects
only_useful = Graph()
only_useful += ((s, p, o) for s, p, o in filtered_graph if s in subjects_to_keep)

# Print statistics
print(f"Original graph has {len(filtered_graph)} triples.")
print(f"Filtered graph has {len(only_useful)} triples.")
only_useful.serialize("filtered_POIs_only.ttl", format="turtle")

Original graph has 76981 triples.
Filtered graph has 6099 triples.


<Graph identifier=Nf0e32a2334144e9d9d796d490d0e3efa (<class 'rdflib.graph.Graph'>)>

In [ ]:
for s, p, o in only_useful:
    print(s, p, o)

In [ ]:
# Define the target subject
target_subject = URIRef("http://poimilano.disco.unimib.it/resource/point-of-interest/118773")

# Print all triples where the subject matches
for s, p, o in only_useful:
    if s == target_subject:
        print(s, p, o)

In [ ]:
import pandas as pd
# Define the target predicates
target_predicates = [
    URIRef("http://www.wikidata.org/prop/direct/P31"),  # TYPE
    URIRef("http://www.wikidata.org/prop/direct/P625"),  # COORDINATES
    URIRef("http://www.wikidata.org/prop/direct/P669"),   # PLACE
    URIRef("http://poimilano.disco.unimib.it/property/wikidataEntity") #WIKIDATA
]

# Initialize an empty list to store the rows for the DataFrame
rows = []

# Step 1: Loop through the graph and extract the required data
for subject in only_useful.subjects():  # Loop through all subjects in the graph
    # Initialize a dictionary for this row
    row = {"POI": str(subject)}

    # Extract the objects for each target predicate
    for predicate in target_predicates:
        # Retrieve the matching triples for this predicate
        matching_triples = [obj for s, p, obj in only_useful.triples((subject, predicate, None))]

        # If a matching triple is found, add the object to the row dictionary
        if matching_triples:
            if predicate == target_predicates[0]:
                row["TYPE"] = str(matching_triples[0])  # TYPE for P31
            elif predicate == target_predicates[1]:
                row["COORDINATES"] = str(matching_triples[0])  # COORDINATES for P625
            elif predicate == target_predicates[2]:
                row["PLACE"] = str(matching_triples[0])  # PLACE for P669
            elif predicate == target_predicates[3]:
                row["WIKIDATA"] = str(matching_triples[0]) #wikidata
        else:
            # If no matching object, set as None
            if predicate == target_predicates[0]:
                row["TYPE"] = None
            elif predicate == target_predicates[1]:
                row["COORDINATES"] = None
            elif predicate == target_predicates[2]:
                row["PLACE"] = None
            elif predicate == target_predicates[3]:
                row["WIKIDATA"] = None

    # Append the row to the list
    rows.append(row)

# Step 2: Create a DataFrame from the rows
df = pd.DataFrame(rows)

df.drop_duplicates(inplace=True)
# Extract the characters after "point-of-interest/"
df['POI'] = df['POI'].str.split('point-of-interest/').str[1]
# Extract the characters after "class/"
df['TYPE'] = df['TYPE'].str.split('class/').str[1]
df['PLACE'] = df['PLACE'].str.split('address/').str[1]
# Split the coordinates into latitude and longitude
df[['LATITUDE', 'LONGITUDE']] = df['COORDINATES'].str.extract(r'POINT\(([^ ]+) ([^)]+)\)')
df = df.drop(columns=['COORDINATES'])
df['TYPE'] = df['TYPE'].replace('StazioneSotterranea', 'FermataDellaMetropolitana')
df['TYPE'] = df['TYPE'].replace('MonumentoVotivo', 'Monumento')
df['TYPE'] = df['TYPE'].replace('MonumentoCommemorativo', 'Monumento')
df['TYPE'] = df['TYPE'].replace('MonumentoDiAvvertimento', 'Monumento')
# Example list of allowed values in the 'TYPE' column

allowed_types = [
    "FermataDellaMetropolitana",
    "Museo",
    "MonumentoCommemorativo",
    "Monumento",
    "Scultura",
    "GalleriaDArte",
    "PiantaMonumentale"
    "SalaDegliArchivi"
    "Statua",
    "TargaCommemorativa",
    "Teatro",
    "ParcoEPiazza"
]

# Assuming your DataFrame is named df
df = df[df['TYPE'].isin(allowed_types)]

In [ ]:
import pandas as pd
import numpy as np

# Define the function to compute the Haversine distance
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return R * c  # Distance in km

# Ensure LATITUDE and LONGITUDE are float
df["LATITUDE"] = df["LATITUDE"].astype(float)
df["LONGITUDE"] = df["LONGITUDE"].astype(float)

# Duomo coordinates
duomo_lat, duomo_lon = 45.4642570, 9.1901143

# Compute distances
df["DISTANCE"] = df.apply(lambda row: haversine(duomo_lat, duomo_lon, row["LATITUDE"], row["LONGITUDE"]), axis=1)

# Filter POIs within 3 km
df_filtered = df[df["DISTANCE"] < 2]

# Display the filtered DataFrame
df_filtered


,POI,TYPE,PLACE,WIKIDATA,LATITUDE,LONGITUDE,DISTANCE
1,115565,Museo,corso-magenta,http://www.wikidata.org/entity/Q3972975,45.465601,9.172526,1.379760
2,68979,FermataDellaMetropolitana,corso-lodi,http://www.wikidata.org/entity/Q1088096,45.451667,9.203056,1.725899
6,116047,Monumento,None,http://www.wikidata.org/entity/Q69545741,45.465771,9.188419,0.214058
8,131866,Teatro,via-mercato,http://www.wikidata.org/entity/Q3982314,45.471461,9.183917,0.935540
11,125981,TargaCommemorativa,piazza-castello,http://www.wikidata.org/entity/Q56642065,45.469470,9.180030,0.976956
...,...,...,...,...,...,...,...
1270,140432,Museo,viale-legione-lombarda,http://www.wikidata.org/entity/Q3604681,45.474016,9.180712,1.309628
1295,107607,Museo,palazzo-reale,http://www.wikidata.org/entity/Q16337697,45.463246,9.191625,0.162845
1512,106482,TargaCommemorativa,via-carlo-giuseppe-merlo,http://www.wikidata.org/entity/Q56642074,45.462840,9.195770,0.468375
1662,117353,Museo,via-giuseppe-antonio-sassi,http://www.wikidata.org/entity/Q55388576,45.466260,9.170548,1.542059


In [ ]:
#GET UNIQUE WIKI IDS FOR QUERY
unique_wiki_ids = df_filtered['WIKIDATA'].str.split('/entity/', expand=True)[1].unique()

In [ ]:
import pandas as pd
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

# Filter out nan values and convert to list
wikidata_ids = [id for id in unique_wiki_ids if pd.notna(id)]

# SPARQL endpoint
endpoint_url = "https://query.wikidata.org/sparql"

# Query template for getting labels
query_template = """
SELECT ?rdfsLabel
WHERE {{
  wd:{entity_id} rdfs:label ?rdfsLabel.
  FILTER(LANG(?rdfsLabel) = "en")
}}
"""

def get_results(endpoint_url, query):
    """Fetch results from the SPARQL endpoint."""
    user_agent = f"WDQS-example Python/{sys.version_info[0]}.{sys.version_info[1]}"
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

# Create a list to store results
results_list = []

# Query each Wikidata ID
for entity_id in wikidata_ids:
    query = query_template.format(entity_id=entity_id)
    try:
        results = get_results(endpoint_url, query)
        if results["results"]["bindings"]:
            label = results["results"]["bindings"][0]["rdfsLabel"]["value"]
        else:
            label = None
    except Exception as e:
        print(f"Error querying {entity_id}: {e}")
        label = None

    results_list.append({
        'wikidata_id': entity_id,
        'label': label
    })

# Create DataFrame
labels_df = pd.DataFrame(results_list)

# Sort by wikidata_id for better readability
labels_df = labels_df.sort_values('wikidata_id').reset_index(drop=True)

print("Labels DataFrame:")
print(labels_df)

In [ ]:
#CLEAN WIKIDATA
df_filtered['WIKIDATA'] = df_filtered['WIKIDATA'].str.split('/').str[-1]

In [ ]:
#MERGE
labelled_df = pd.merge(df_filtered, labels_df, left_on='WIKIDATA', right_on='wikidata_id', how='left', suffixes=('_result', '_label_'))
#DROP NA
labelled_df.dropna(inplace=True)
#REMOVE SPACE
labelled_df['label'] = labelled_df['label'].str.replace(' ', '')
labelled_df

In [ ]:
# Group the DataFrame by the 'TYPE' column and count the occurrences of each type.
type_counts = labelled_df.groupby('TYPE').size()

# Display the type counts.
type_counts

In [ ]:
labelled_df.to_csv('FEB_15_POI_DATA_CENTERED_2KM_DUMO.csv')